### Testar o modelo criado

In [1]:
import pandas as pd
import string # pré-processamento
import spacy
import random 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from spacy.lang.pt.stop_words import STOP_WORDS

In [2]:
PLN = spacy.load('pt_core_news_sm')

In [3]:
def pre_processamento(texto: str) -> list:
    """
    Percorrer os valores de uma str e adiciona o lema destes na lista
    """ 
    texto = texto.lower()
    documento = PLN(text=texto)

    lista = []
    # Tokenizacao
    for token in documento:
        lista.append(token.lemma_)
    
    # Remocao de Stop word
    lista = [palavra for palavra in lista if palavra not in STOP_WORDS and palavra not in string.punctuation]
    
    # Formato de texto -- # '59'.isdigit() > True
    lista = ' '.join( str(elemento) for elemento in lista if not elemento.isdigit() )
    
    return lista

In [4]:
modelo_carregado = spacy.load("trained_model")
modelo_carregado

In [5]:
texto_alegre = 'Eu adoro a cor dos seus olhos'
texto_triste = 'Estou apavorado com o mestrado'

texto_alegre = pre_processamento(texto=texto_alegre)
texto_triste = pre_processamento(texto=texto_triste)
texto_alegre, texto_triste

('adorar cor de o olho', 'apavorado mestrado')

In [6]:
previsao1, previsao2 = modelo_carregado(texto_alegre), modelo_carregado(texto_triste)

print(f"{previsao1.cats=}")
print(f"{previsao2.cats=}")

previsao1.cats={'ALEGRIA': 1.0, 'MEDO': 1.721377529406709e-08}
previsao2.cats={'ALEGRIA': 0.02342686802148819, 'MEDO': 0.9765731692314148}


## Avaliação do Modelo

In [7]:
# Carregamento da base e pré-processamento
db = pd.read_csv('base_treinamento.txt', encoding='utf-8')
db['texto'] = db['texto'].apply(pre_processamento)
db

,texto,emocao
0,trabalho agradável,alegria
1,gostar ficar em o aconchego,alegria
2,fiz adesão a o curso hoje,alegria
3,admirar,alegria
4,adorar,alegria
...,...,...
191,chocado amedrontar assassinato brutal,medo
192,preciso afugentar ímpeto medo de o inferno,medo
193,político força afugentar amedrontar,medo
194,objetivo de isso amedrontar,medo


In [8]:
previsoes = []

for texto in db['texto']:
    previsao = modelo_carregado(texto)
    previsoes.append(previsao.cats)

previsoes

[{'ALEGRIA': 1.0, 'MEDO': 5.920351320298778e-09},
 {'ALEGRIA': 1.0, 'MEDO': 2.511861296738971e-08},
 {'ALEGRIA': 1.0, 'MEDO': 2.5298726669120697e-08},
 {'ALEGRIA': 1.0, 'MEDO': 2.6770619498961423e-08},
 {'ALEGRIA': 1.0, 'MEDO': 7.049444583628883e-09},
 {'ALEGRIA': 1.0, 'MEDO': 3.2818135764500767e-08},
 {'ALEGRIA': 1.0, 'MEDO': 1.721377529406709e-08},
 {'ALEGRIA': 1.0, 'MEDO': 3.2386953563445786e-08},
 {'ALEGRIA': 1.0, 'MEDO': 5.626999755037332e-09},
 {'ALEGRIA': 1.0, 'MEDO': 4.165410416590021e-08},
 {'ALEGRIA': 1.0, 'MEDO': 1.0698840924305841e-08},
 {'ALEGRIA': 1.0, 'MEDO': 5.423698379303232e-09},
 {'ALEGRIA': 1.0, 'MEDO': 2.1836534802588403e-09},
 {'ALEGRIA': 1.0, 'MEDO': 1.6920157719368945e-08},
 {'ALEGRIA': 0.9999998807907104, 'MEDO': 1.4140091764147655e-07},
 {'ALEGRIA': 1.0, 'MEDO': 2.3061877740815362e-08},
 {'ALEGRIA': 1.0, 'MEDO': 2.071314675333724e-08},
 {'ALEGRIA': 1.0, 'MEDO': 7.851483019294392e-09},
 {'ALEGRIA': 1.0, 'MEDO': 2.8430669374301942e-08},
 {'ALEGRIA': 1.0, 'MEDO':

In [9]:
previsoes_final = []

for previsao in previsoes:
    if previsao['ALEGRIA'] > previsao['MEDO']:
        previsoes_final.append('alegria')
    else:
        previsoes_final.append('medo')

previsoes_final = np.array(previsoes_final)

In [10]:
qtd_acertos = sum(db['emocao'] == previsoes_final)
qtd_acertos

# Fizemos a previsao com a propria base de treinamento

196

## Avaliação da base de teste

In [11]:
db_teste = pd.read_csv("base_teste.txt", encoding='utf-8')
db_teste

,texto,emocao
0,não precisei pagar o ingresso,alegria
1,se eu ajeitar tudo fica bem,alegria
2,minha fortuna ultrapassará a sua,alegria
3,sou muito afortunado,alegria
4,é benéfico para todos esta nova medida,alegria
...,...,...
79,fui notificado e estou com medo de perder a gu...,medo
80,estou angustiada com meus filhos na rua,medo
81,e abominável o que fazem com os animais,medo
82,foi terrível o tigre quase o matou,medo


In [12]:
db_teste['texto'] = db_teste['texto'].apply(pre_processamento)
db_teste

,texto,emocao
0,precisar pagar ingresso,alegria
1,ajeitar ficar,alegria
2,fortuna ultrapassar,alegria
3,afortunar,alegria
4,benéfico medida,alegria
...,...,...
79,notificar medo perder guarda de o filha,medo
80,angustiada filho em o rua,medo
81,abominável animal,medo
82,terrível tigre quase matar,medo


In [13]:
previsoes = []

for texto in db_teste['texto']:
    previsao = modelo_carregado(texto)
    previsoes.append(previsao.cats)


previsoes_final = []

for previsao in previsoes:
    if previsao['ALEGRIA'] > previsao['MEDO']:
        previsoes_final.append('alegria')
    else:
        previsoes_final.append('medo')

previsoes_final = np.array(previsoes_final)

In [ ]:
sum(previsoes_final == db_teste['emocao'])


47

In [17]:
from sklearn.metrics import confusion_matrix


In [ ]:
cm = confusion_matrix(y_true=db_teste['emocao'], y_pred=previsoes_final)
cm

array([[34, 14],
       [23, 13]])

Essa base de dados de treinamento não é muito boa, serve apenas para mostrar uma visão geral do spaCy.